In [ ]:
import panel as pn

def get_structure_from_url():
    """Get structure label from URL parameter 'id'"""
    from bokeh.io import curdoc
    try:
        name = curdoc().session_context.request.arguments.get('id')[0]
        if isinstance(name, bytes):
            structure_label = name.decode()
    except (TypeError, KeyError, AttributeError):
        structure_label = '05001N2'

    return structure_label


def provenance_link(uuid, label=None):
    """Return pn.HTML representation of provenance link."""
    import os
    
    if label is None:
        label = "Browse provenance\n" + uuid
        
    logo_url = "https://www.materialscloud.org/discover/images/ceede967.aiida-logo-128.png"
    explore_url = os.getenv('EXPLORE_URL', "https://dev-www.materialscloud.org/explore/curated-cofs")
    return pn.pane.HTML("<a href='{url}/details/{uuid}' target='_blank'><img src={logo_url} title='{label}' style='width: 20px;  height: auto;'></a>".
        format(
        url=explore_url, uuid=uuid, label=label, logo_url=logo_url), align='center')

def get_title(text, uuid=None):
    """Return pn.Row representation of title.
    
    Includes provenance link, if uuid is specified.
    """
    title = pn.Row(pn.pane.Markdown('#### '+text), align='center')
    
    if uuid is not None:
        title.append(provenance_link(uuid))
        
    return title

In [ ]:
import param
from query import get_data_aiida

pn.extension()

class DetailView(param.Parameterized):
    
#     x = param.Selector(objects=plot_dict, default='PE', doc="abcd")
#     y = param.Selector(objects=plot_dict, default='Pur')
#     clr = param.Selector(objects=OrderedDict(plot_dict, **{'Bond type': 'bond_type'}), default='Td')
    msg = pn.pane.HTML("")
    label=get_structure_from_url()
    _plot = None  # reference to current plot
    
    @param.depends()
    def iso_plot(self):
        from detail.plot_isotherm import plot_isotherm
        label = self.label
        pm_co2 = get_data_aiida(structure_label=label, link_label='iso_co2')
        pm_n2 = get_data_aiida(structure_label=label, link_label='iso_n2')
        mpl_fig = plot_isotherm(pm_co2=pm_co2, pm_n2=pm_n2, label=label)

        column = pn.Column(get_title("Isotherms for " + self.label, uuid=pm_co2.uuid))
        column.append(pn.pane.Matplotlib(mpl_fig, dpi=150, width=700))

        return column

    @param.depends()
    def ener_plot(self):
        from detail.plot_cp2k import plot_energy_steps
        steps_node=get_data_aiida(structure_label=self.label, link_label='cp2k_stepsfile')
        steps_file = steps_node.get_file_abs_path()

        column = pn.Column(get_title("Robust cell optimization of " + self.label, uuid=steps_node.uuid))
        column.append(pn.pane.Bokeh(plot_energy_steps(steps_file, self.label)))
        return column

    @param.depends()
    def structure_ref(self):
        from detail.structure import structure_jsmol
        cif_node = get_data_aiida(structure_label=self.label, link_label='ref_cif_qeq')
        fpath = cif_node.get_file_abs_path()
        with open(fpath, 'r') as handle:
            cif_str = handle.read()
            
        column = pn.Column(get_title("Reported structure for " + self.label, uuid=cif_node.uuid))
        column.append(pn.pane.Bokeh(structure_jsmol(cif_str=cif_str)))
        return column
  
    @param.depends()
    def structure_opt(self):
        from detail.structure import structure_jsmol
        cif_node = get_data_aiida(structure_label=self.label, link_label='opt_cif_ddec')
        fpath = cif_node.get_file_abs_path()
        with open(fpath, 'r') as handle:
            cif_str = handle.read()
            
        column = pn.Column(get_title("DFT-optimized structure for " + self.label, uuid=cif_node.uuid))
        column.append(pn.pane.Bokeh(structure_jsmol(cif_str=cif_str)))
        return column
    
    @param.depends()
    def process(self):
        from detail.process import print_process
        process_node =  get_data_aiida(structure_label=self.label, link_label='opt_out_pe')
        process_dict = process_node.get_dict()
        column = pn.Column(get_title("Process sketch for " + self.label, uuid=process_node.uuid))
        column.append(pn.pane.PNG('detail/process.png', width=800))
        column.append(pn.pane.Markdown(print_process(process_node)))
        return column
        
explorer = DetailView()

# gspec = pn.GridSpec(sizing_mode='stretch_both', max_width=900)
# #gspec[0, 0] = explorer.structure
# gspec[1, 0] = explorer.ener_plot
# gspec[2, 0] = explorer.iso_plot
# gspec.servable()

tabs = pn.Tabs()
tabs.extend([
    ('Process evaluation', explorer.process),
    ('Reported structure', explorer.structure_ref),
    ('DFT-optimization energy', explorer.ener_plot),
    ('DFT-optimized structure', explorer.structure_opt),
    ('Isotherms', explorer.iso_plot),
])

tabs.servable()